In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("humairmunir/lung-cancer-risk-dataset")

print("Path to dataset files:", path)

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/morning/.cache/kagglehub/datasets/humairmunir/lung-cancer-risk-dataset/versions/3


In [2]:
import os
os.listdir(path)

['dataseter.csv']

In [3]:
import pandas as pd
import fairtl_statisticaltest as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
df = pd.read_csv(f'{path}/dataseter.csv')
df.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,M,65,Yes,Yes,Yes,No,No,Yes,No,No,No,No,No,No,Yes,NO
1,F,55,Yes,No,No,Yes,Yes,No,No,No,Yes,Yes,Yes,No,No,NO
2,F,78,No,No,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,Yes,Yes,YES
3,M,60,No,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,Yes,No,No,YES
4,F,80,Yes,Yes,No,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Yes,No,NO


In [5]:
df.shape

(3000, 16)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   GENDER                 3000 non-null   object
 1   AGE                    3000 non-null   int64 
 2   SMOKING                3000 non-null   object
 3   YELLOW_FINGERS         3000 non-null   object
 4   ANXIETY                3000 non-null   object
 5   PEER_PRESSURE          3000 non-null   object
 6   CHRONIC_DISEASE        3000 non-null   object
 7   FATIGUE                3000 non-null   object
 8   ALLERGY                3000 non-null   object
 9   WHEEZING               3000 non-null   object
 10  ALCOHOL_CONSUMING      3000 non-null   object
 11  COUGHING               3000 non-null   object
 12  SHORTNESS_OF_BREATH    3000 non-null   object
 13  SWALLOWING_DIFFICULTY  3000 non-null   object
 14  CHEST_PAIN             3000 non-null   object
 15  LUNG_CANCER          

In [7]:
df.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,M,65,Yes,Yes,Yes,No,No,Yes,No,No,No,No,No,No,Yes,NO
1,F,55,Yes,No,No,Yes,Yes,No,No,No,Yes,Yes,Yes,No,No,NO
2,F,78,No,No,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,Yes,Yes,YES
3,M,60,No,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,Yes,No,No,YES
4,F,80,Yes,Yes,No,Yes,Yes,No,Yes,No,Yes,Yes,Yes,Yes,No,NO


In [8]:
df['LUNG_CANCER'].unique()

array(['NO', 'YES'], dtype=object)

In [9]:
df['GENDER'] = df['GENDER'].map({'F': 0, 'M': 1})
df['SMOKING'] = df['SMOKING'].map({'No': 0, 'Yes': 1})
df['YELLOW_FINGERS'] = df['YELLOW_FINGERS'].map({'No': 0, 'Yes': 1})
df['ANXIETY'] = df['ANXIETY'].map({'No': 0, 'Yes': 1})
df['PEER_PRESSURE'] = df['PEER_PRESSURE'].map({'No': 0, 'Yes': 1})
df['CHRONIC_DISEASE'] = df['CHRONIC_DISEASE'].map({'No': 0, 'Yes': 1})
df['FATIGUE'] = df['FATIGUE'].map({'No': 0, 'Yes': 1})
df['ALLERGY'] = df['ALLERGY'].map({'No': 0, 'Yes': 1})
df['WHEEZING'] = df['WHEEZING'].map({'No': 0, 'Yes': 1})
df['ALCOHOL_CONSUMING'] = df['ALCOHOL_CONSUMING'].map({'No': 0, 'Yes': 1})
df['COUGHING'] = df['COUGHING'].map({'No': 0, 'Yes': 1})
df['SHORTNESS_OF_BREATH'] = df['SHORTNESS_OF_BREATH'].map({'No': 0, 'Yes': 1})
df['SWALLOWING_DIFFICULTY'] = df['SWALLOWING_DIFFICULTY'].map({'No': 0, 'Yes': 1})
df['CHEST_PAIN'] = df['CHEST_PAIN'].map({'No': 0, 'Yes': 1})
df['LUNG_CANCER'] = df['LUNG_CANCER'].map({'NO': 0, 'YES': 1})


In [10]:
df.head()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,1,65,1,1,1,0,0,1,0,0,0,0,0,0,1,0
1,0,55,1,0,0,1,1,0,0,0,1,1,1,0,0,0
2,0,78,0,0,1,1,1,0,1,0,1,1,0,1,1,1
3,1,60,0,1,1,1,0,1,0,1,1,0,1,0,0,1
4,0,80,1,1,0,1,1,0,1,0,1,1,1,1,0,0


In [11]:
y = df['LUNG_CANCER'].values
X = df.drop('LUNG_CANCER', axis=1)

In [12]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(3000, 15)
(3000, 15)


In [13]:
# build mask
gender_0_mask = df['GENDER'] == 0
gender_1_mask = df['GENDER'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  1486
Male:  1514


In [14]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [15]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 9 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 10 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
    Fold   Group   SVM_TPR   SVM_TNR   SVM_FPR   SVM_FNR  SVM_TP  SVM_TN  \
0      1  Female  0.610390  0.402778  0.597222  0.389610      47      29   
1      2  Female  0.439024  0.582090  0.417910  0.560976      36      39   
2      3  Female  0.476190  0.430769  0.569231  0.523810      40      28   
3      4  Female  0.636364  0.472222  0.527778  0.363636      49      34   
4      5  Female  0.556962  0.414286  0.585714  0.443038      44      29   
5      6  Female  0.543210  0.426471  0.573529  0.456790      44      29   
6      7  Female  0.617647  0.437500  0.562500  0.382353      42      35   
7      8  Female  0.597403  0.436620  0.563380  0.402597      46      31   
8      

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [16]:
results_df = pd.concat(results_list, ignore_index=True)

In [17]:
result_path = './results/K24_result.xlsx'
results_df.to_excel(result_path, index=False)

In [18]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,0.610390,0.402778,0.597222,0.389610,47,29,43,30,...,36,34,0.688312,0.361111,0.638889,0.311688,53,26,46,24
1,2,Female,0.439024,0.582090,0.417910,0.560976,36,39,28,46,...,21,49,0.573171,0.388060,0.611940,0.426829,47,26,41,35
2,3,Female,0.476190,0.430769,0.569231,0.523810,40,28,37,44,...,34,43,0.583333,0.476923,0.523077,0.416667,49,31,34,35
3,4,Female,0.636364,0.472222,0.527778,0.363636,49,34,38,28,...,33,32,0.545455,0.416667,0.583333,0.454545,42,30,42,35
4,5,Female,0.556962,0.414286,0.585714,0.443038,44,29,41,35,...,38,42,0.620253,0.357143,0.642857,0.379747,49,25,45,30


In [19]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: TtestResult(statistic=np.float64(3.047466410706185), pvalue=np.float64(0.006930639842612134), df=np.float64(18.0))
SVM - FPR: TtestResult(statistic=np.float64(3.9349117053093075), pvalue=np.float64(0.0009708736050622259), df=np.float64(18.0))
SVM - FN/FP: TtestResult(statistic=np.float64(-2.1932006725558124), pvalue=np.float64(0.041669062568869694), df=np.float64(18.0))
DT -TPR: TtestResult(statistic=np.float64(1.4666302511388734), pvalue=np.float64(0.15973043581490498), df=np.float64(18.0))
DT - FPR: TtestResult(statistic=np.float64(0.44302415053772787), pvalue=np.float64(0.6630266479340594), df=np.float64(18.0))
DT - FN/FP: TtestResult(statistic=np.float64(-0.12303709668076478), pvalue=np.float64(0.9034411080146717), df=np.float64(18.0))
RF -TPR: TtestResult(statistic=np.float64(3.139185221839213), pvalue=np.float64(0.0056712257671880095), df=np.float64(18.0))
RF - FPR: TtestResult(statistic=np.float64(1.15899963930726), pvalue=np.float64(0.26160154467914015), df=np.float64